## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-11-10-05-49-41 +0000,bbc,Waves smash Philippines coastline as typhoon hits,https://www.bbc.com/news/videos/c709v1l441qo?a...
1,2025-11-10-05-48-01 +0000,nyt,Six Takeaways from the Senate Deal to End the ...,https://www.nytimes.com/2025/11/10/us/politics...
2,2025-11-10-05-37-57 +0000,missionlocal,"S.F. police shoot man, dog near Union Square a...",https://missionlocal.org/2025/11/sf-police-sho...
3,2025-11-10-05-33-41 +0000,startribune,Analysis: Vikings’ fits and (false) starts on ...,https://www.startribune.com/minnesota-vikings-...
4,2025-11-10-05-28-22 +0000,nypost,South Carolina mother on maternity leave nabs ...,https://nypost.com/2025/11/10/us-news/south-ca...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2366/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
90,trump,28
11,shutdown,18
57,new,15
464,bbc,8
4,typhoon,8


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
126,2025-11-09-18-46-21 +0000,nypost,SNAP thrown into chaos after Trump admin tells...,https://nypost.com/2025/11/09/us-news/snap-thr...,79
11,2025-11-10-05-02-42 +0000,nyt,Trump Renews Attacks on Obamacare in New Push ...,https://www.nytimes.com/2025/11/09/us/politics...,74
82,2025-11-09-22-28-52 +0000,nypost,Senate eyes vote on potential breakthrough to ...,https://nypost.com/2025/11/09/us-news/senate-e...,63
69,2025-11-09-23-27-29 +0000,nypost,ICE received more than 200K job applications s...,https://nypost.com/2025/11/09/us-news/ice-rece...,59
17,2025-11-10-04-14-40 +0000,nyt,Sean Duffy Juggles Shutdown Roles on Flight Sa...,https://www.nytimes.com/2025/11/09/us/politics...,58


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
126,79,2025-11-09-18-46-21 +0000,nypost,SNAP thrown into chaos after Trump admin tells...,https://nypost.com/2025/11/09/us-news/snap-thr...
149,38,2025-11-09-14-46-42 +0000,bbc,BBC on Luzon Island as super typhoon hits Phil...,https://www.bbc.com/news/videos/c20p2m83p5go?a...
23,37,2025-11-10-03-56-00 +0000,wsj,The Senate late Sunday cleared a critical proc...,https://www.wsj.com/politics/policy/republican...
6,34,2025-11-10-05-22-38 +0000,wapo,Special counsel indicts former South Korean pr...,https://www.washingtonpost.com/world/2025/11/1...
60,30,2025-11-10-00-09-50 +0000,nypost,Brooklyn dad says goodbye to hero son Itay Che...,https://nypost.com/2025/11/09/world-news/nyc-d...
142,26,2025-11-09-15-33-10 +0000,nypost,Lefty Gen-Z NYC Councilman Chi Osse gears up f...,https://nypost.com/2025/11/09/us-news/nyc-coun...
141,24,2025-11-09-15-56-32 +0000,nypost,Transportation secretary gives dire warning on...,https://nypost.com/2025/11/09/us-news/transpor...
57,23,2025-11-10-00-12-41 +0000,latimes,Here's how many flights have been canceled out...,https://www.latimes.com/california/story/2025-...
78,23,2025-11-09-22-53-45 +0000,nyt,Flight Cancellations and Delays May Get Worse ...,https://www.nytimes.com/2025/11/09/business/ai...
125,21,2025-11-09-18-48-33 +0000,cbc,"BBC director, head of news resign after critic...",https://www.cbc.ca/news/world/bbc-director-res...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
